In [1]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab, Vocab
from torchtext.utils import download_from_url, extract_archive

import io
from collections import Counter
from tqdm import tqdm
import random
from typing import Tuple
import math
import time
import copy

In [2]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

In [3]:
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [4]:
def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in tqdm(f):
            counter.update(tokenizer(string_))
    base_vocab = vocab(
        ordered_dict = counter, 
        specials=['<unk>', '<pad>', '<bos>', '<eos>'],
        min_freq = 2, 
    )
    base_vocab.set_default_index(base_vocab['<unk>'])
    return Vocab(base_vocab)

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

29000it [00:01, 24210.07it/s]
29000it [00:00, 35350.05it/s]


In [5]:
print(f"de_vocab type: {type(de_vocab)}")
print(f"en_vocab type: {type(en_vocab)}")

print(f"de_vocab length: {len(de_vocab)}")
print(f"en_vocab length: {len(en_vocab)}")

de_vocab type: <class 'torchtext.vocab.vocab.Vocab'>
en_vocab type: <class 'torchtext.vocab.vocab.Vocab'>
de_vocab length: 8015
en_vocab length: 6192


In [6]:
def data_process(filepaths):
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_de, raw_en) in tqdm(zip(raw_de_iter, raw_en_iter)):
        de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)], dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)], dtype=torch.long)
        data.append((de_tensor_, en_tensor_))
    
    return data

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

29000it [00:02, 12932.02it/s]
1014it [00:00, 9664.14it/s]
1000it [00:00, 10276.78it/s]


In [7]:
print(f"Train Data Type: {type(train_data)}")
print(f"Val Data Type: {type(val_data)}")
print(f"Test Data Type: {type(test_data)}")

Train Data Type: <class 'list'>
Val Data Type: <class 'list'>
Test Data Type: <class 'list'>


In [8]:
print(f"#Training Samples: {len(train_data)}")
print(f"#Validation Samples: {len(val_data)}")
print(f"#Testing Samples: {len(test_data)}")

#Training Samples: 29000
#Validation Samples: 1014
#Testing Samples: 1000


In [9]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

def generate_batch(data_batch):
    de_batch, en_batch = [], []
    for (de_item, en_item) in data_batch:
        de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
    de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    return de_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)

In [10]:
print(f"#Training Batches: {len(train_iter)}")
print(f"#Validation Batches: {len(valid_iter)}")
print(f"#Testing Batches: {len(test_iter)}")

#Training Batches: 227
#Validation Batches: 8
#Testing Batches: 8


In [11]:
for sample_de, sample_en in train_iter:
    print(sample_de.shape)
    print(sample_en.shape)
    break

torch.Size([27, 128])
torch.Size([27, 128])


In [12]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        return hidden, cell

In [13]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))        
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):       
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        hidden, cell = self.encoder(src)
        
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
                    
            top1 = output.argmax(1) 
            
            input = trg[t] if teacher_force else top1
        
        return outputs
    
    def single_predict(self, src, max_len = 100):
        if self.training:
            self.training = False

        print(src.shape)
        hidden, cell = self.encoder(src)

        input = copy.deepcopy(src[0, :])

        outputs = []

        while input.item() != EOS_IDX and len(outputs) < 100:
            output, hidden, cell = self.decoder(input, hidden, cell)

            input = output.argmax(1) 

            outputs.append(input.item())

        return outputs

In [15]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_EPOCHS = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu', index = 0)

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters())

PAD_IDX = en_vocab['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [16]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(8015, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(6192, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=6192, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,169,904 trainable parameters


In [18]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    model.train()
    
    total_train_loss = 0
    for train_src, train_trg in tqdm(train_iter):
        train_src, train_trg = train_src.to(device), train_trg.to(device)
        
        train_output = model(train_src, train_trg)
        train_output_dim = train_output.shape[-1]
        train_output = train_output[1:].view(-1, train_output_dim)
        train_trg = train_trg[1:].view(-1)
        
        train_loss = criterion(train_output, train_trg)

        optimizer.zero_grad()
        
        train_loss.backward()
        
        optimizer.step()
        
        total_train_loss += train_loss.item()

    model.eval()

    total_valid_loss = 0
    with torch.no_grad():
        for valid_src, valid_trg in tqdm(valid_iter):
            valid_src, valid_trg = valid_src.to(device), valid_trg.to(device)

            valid_output = model(valid_src, valid_trg, 0)
            valid_output_dim = valid_output.shape[-1]
            valid_output = valid_output[1:].view(-1, valid_output_dim)
            valid_trg = valid_trg[1:].view(-1)

            valid_loss = criterion(valid_output, valid_trg)
            
            total_valid_loss += valid_loss.item()

    mean_train_loss = total_train_loss/len(train_iter)
    mean_valid_loss = total_valid_loss/len(valid_iter)
    

    print(f"Epoch: {epoch} - Mean Train Loss: {mean_train_loss} - Mean Valid Loss: {mean_valid_loss}")

100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 32.63it/s]


Epoch: 0 - Mean Train Loss: 4.913150900786143 - Mean Valid Loss: 4.89428174495697


100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 31.58it/s]


Epoch: 1 - Mean Train Loss: 4.2851835977663555 - Mean Valid Loss: 4.597559452056885


100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 32.22it/s]


...

Epoch: 97 - Mean Train Loss: 0.3652272792245848 - Mean Valid Loss: 5.849691212177277


100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 33.08it/s]


Epoch: 98 - Mean Train Loss: 0.3563655014211386 - Mean Valid Loss: 5.847384810447693


100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 30.95it/s]

Epoch: 99 - Mean Train Loss: 0.3569537969675358 - Mean Valid Loss: 5.841828465461731


In [19]:
model.eval()

total_test_loss = 0
with torch.no_grad():
    for test_src, test_trg in tqdm(test_iter):
        test_src, test_trg = test_src.to(device), test_trg.to(device)

        test_output = model(test_src, test_trg, 0)
        test_output_dim = test_output.shape[-1]
        test_output = test_output[1:].view(-1, test_output_dim)
        test_trg = test_trg[1:].view(-1)

        test_loss = criterion(test_output, test_trg)
        
        total_test_loss += test_loss.item()

mean_test_loss = total_test_loss/len(test_iter)

print(f"Test Loss: {mean_test_loss}")

100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 32.91it/s]

Test Loss: 5.794943273067474


In [27]:
for case, (idx_src, idx_trg) in enumerate(test_data[:10]):
    token_src = de_vocab.lookup_tokens(idx_src.numpy().tolist())
    token_trg = en_vocab.lookup_tokens(idx_trg.numpy().tolist())
    print(f"CASE: {case}")
    joined_token_src = " ".join(token_src)
    joined_token_trg = " ".join(token_trg)
    print(f"\tSOURCE: {joined_token_src}")
    print(f"\tTARGET: {joined_token_trg}")

    processed_idx_src = torch.cat([torch.tensor([BOS_IDX]), idx_src, torch.tensor([EOS_IDX])], dim=0)
    processed_idx_trg = torch.cat([torch.tensor([BOS_IDX]), idx_trg, torch.tensor([EOS_IDX])], dim=0)

    batch_idx_src = processed_idx_src.unsqueeze(1)
    batch_idx_trg = processed_idx_trg.unsqueeze(1)

    translated = model.single_predict(idx_src.to(device).unsqueeze(1))

    token_translated = en_vocab.lookup_tokens(translated)
    join_token_translated = " ".join(token_translated)
    print(f"\tPREDICT: {join_token_translated}")

CASE: 0
	SOURCE: Ein Mann mit einem orangefarbenen Hut , der etwas <unk> . 

	TARGET: A man in an orange hat starring at something . 

torch.Size([12, 1])
	PREDICT: A man wearing an orange hat - up . 
 <eos>
CASE: 1
	SOURCE: Ein Boston Terrier läuft über <unk> Gras vor einem weißen Zaun . 

	TARGET: A Boston Terrier is running on lush green grass in front of a white fence . 

torch.Size([13, 1])
	PREDICT: A brown dog runs across the grass of a fence . 
 <eos>
CASE: 2
	SOURCE: Ein Mädchen in einem Karateanzug bricht ein Brett mit einem Tritt . 

	TARGET: A girl in karate uniform breaking a stick with a front kick . 

torch.Size([13, 1])
	PREDICT: A girl in a karate costume does a silly face . 
 <eos>
CASE: 3
	SOURCE: Fünf Leute in Winterjacken und mit Helmen stehen im Schnee mit <unk> im Hintergrund . 

	TARGET: Five people wearing winter jackets and helmets stand in the snow , with <unk> in the background . 

torch.Size([16, 1])
	PREDICT: Five people wearing a sports suits and helmets 